# Chapter 2, Variables

In [4]:
let languages = "OCaml;Go;Rust;Elm" ;;
let language_list = String.split languages ~on:';' in
    String.concat ~sep:"-" language_list ;;

val languages : string = "OCaml;Go;Rust;Elm"


- : Core.String.t = "OCaml-Go-Rust-Elm"


### Pattern matching with let

In [7]:
List.unzip [(1, "uno");(2, "dos");(3, "tres")] ;;
let (ints, strings) = List.unzip [(1, "uno");(2, "dos");(3, "tres")]

- : int Core.List.t * string Core.List.t =
([1; 2; 3], ["uno"; "dos"; "tres"])


val ints : int Core.List.t = [1; 2; 3]
val strings : string Core.List.t = ["uno"; "dos"; "tres"]


  “Tuple and record patterns are irrefutable, but list patterns are not” 

In [9]:
let (first :: rest) = [1;2;3;4]

File "[9]", line 1, characters 4-19:
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
[]


val first : int = 1
val rest : int list = [2; 3; 4]


## Functions

We declare them with `let` the same as variables (after all, they are first citizens).

We have anonymous functions too:

In [10]:
(fun x -> x * 2)

- : int -> int = <fun>


In [11]:
let operations = [(fun x -> x * 2); (fun x -> x + 2)] ;;
List.map ~f:(fun x -> x 2) operations

val operations : (int -> int) list = [<fun>; <fun>]


- : int Core.List.t = [4; 4]


We can assign a name for our function, after all, it is like any other variable or type:

In [1]:
let double = (fun x -> x * 2) ;;
double 10

val double : int -> int = <fun>


- : int = 20


Because this is so common, we have a shortcut for it in the language:

In [2]:
let double x = x * 2 ;;
double 10

val double : int -> int = <fun>


- : int = 20


What about multiple arguments? that is easy, just append the arguments to the function:

In [3]:
let multiply x y = x * y

val multiply : int -> int -> int = <fun>


It is important to notice this is the equivalent to the following expression:

In [5]:
let multiply = (fun x -> (fun y -> x * y))

val multiply : int -> int -> int = <fun>


As you can see, we _curry_ the function arguments, so we can do _partial applications_ easily:

In [10]:
let by_two = multiply 2 ;;

by_two 3

val by_two : int -> int = <fun>


- : int = 6


Creating and consuming currying functions has no penalty, but _partial application of a function_ has a small performance penalty. This is the standard calling convention in OCaml, so stick to it

### Recursive functions

This is probably the most difficult part to fully grasp from OCaml, not recursion at all, but _tail recursion_. Because every time we call a function we occupy a space in the stack and the stack is finite, we can run into the problem that after a while we consume all the available space in the stack. To avoid this we can generate functions that can be discarded immediately and avoid putting the call in the stack.

To create recursive functions in OCaml we have to append the keyword `rec` to the function declaration:

In [12]:
let rec is_even x =
  if x = 0 then true else is_odd (x - 1)
and is_odd x =
  if x = 0 then false else is_even (x - 1)

val is_even : int -> bool = <fun>
val is_odd : int -> bool = <fun>


Notice how we declare mutually recursive functions with the mix of `rec` and `and`

### Infix operators

Operators are nothing more than functions with special treatment. We can use the addition operator putting the symbol in parenthesis:

In [13]:
(+) 2 4

- : int = 6


By convention, every function using the symbols from the following table are treated as prefix, you can define your own operators using those symbols, and the associativity is related to the first symbol in the operator name:

(_Insert associativity table here_)

That means, you can create your own operators for use as infix, for example, let's reconstruct the `apply` operator already in the standard library `|>`

In [14]:
let (|>) x f =
  f x ;;

val ( |> ) : 'a -> ('a -> 'b) -> 'b = <fun>


In [15]:
let path = "/usr/bin:/usr/local/bin:/bin:/sbin" ;;
String.split ~on:':' path
  |> List.dedup ~compare:String.compare
  |> List.iter ~f:print_endline

val path : string = "/usr/bin:/usr/local/bin:/bin:/sbin"


/bin
/sbin
/usr/bin
/usr/local/bin


- : unit = ()


The equivalent without using the `apply` operator is a little more verbose:

In [17]:
let path = "/usr/bin:/usr/local/bin:/bin:/sbin" ;;
let split_path = String.split ~on:':' path in
let deduped_path = List.dedup ~compare:String.compare split_path in
List.iter ~f:print_endline deduped_path

val path : string = "/usr/bin:/usr/local/bin:/bin:/sbin"


/bin
/sbin
/usr/bin
/usr/local/bin


- : unit = ()


The secret here is in part the usage of _partial application_, because if you think about it, we create partial functions in `List.dedup` and `List.iter` and then just apply the last argument using the `|>` operator. Because in the table `|` creates a _left associative_ operator this is possible as well. In other words, it works because we first process what is in the left and the pass to what is in the right. For example, if instead of `|>` we use a _right associative_ operator, like `^>` we will try to pass a function to `List.iter` instead of list, and that will fail.

### Pattern matching and function

Pattern matching is very common when using functions, for example:

In [18]:
let some_or_zero x =
  match x with
  | Some x -> x
  | None -> 0 ;;

val some_or_zero : int option -> int = <fun>


This is so common that we have a way to do it, or a shortcut of sorts:

In [19]:
let some_or_zero = function
  | Some x -> x
  | None -> 0 ;;

val some_or_zero : int option -> int = <fun>


We can do this as well with more than one argument, but the last argument is always the argument in the replaced `match` clause: (notice the signature of the function)

In [20]:
let some_or_default default = function
  | Some x -> x
  | None -> default ;;

val some_or_default : 'a -> 'a option -> 'a = <fun>


In [21]:
List.map ~f:(some_or_default 100) [Some 3; None; Some 4]

- : int Core.List.t = [3; 100; 4]


### Labeled arguments

We had seen this already, for example in `List.map ~f:(some_or_default 100) [Some 3;None]`. The secret is using `~` when naming the argument:

In [22]:
let ratio ~num ~denom = float num /. float denom ;;

val ratio : num:int -> denom:int -> float = <fun>


Notice the signature, it will explicitly mark the type and name of the arguments when using this syntax.

A nice thing when using naming arguments is OCaml support for _name punning_, or in other words, if you have a variable with the same name as the argument that will be the applied argument to the function. For example:

In [23]:
let num = 3 in
let denom = 4 in
ratio ~num ~denom

- : float = 0.75


While the order of arguments with named arguments doesn't matter, it does matter when using partial applications.

### Optional arguments

We can use optional arguments as well with OCaml, just use the `?` operator in the argument name and this will be passed as a `Some|None` type:

In [27]:
let concat ?sep x y =
  let sep = match sep with
  | None -> ""
  | Some x -> x in
  x ^ sep ^ y ;;

val concat : ?sep:string -> string -> string -> string = <fun>


In [28]:
concat "a" "b" ;;
concat ~sep:"+" "a" "b"

- : string = "ab"


- : string = "a+b"


As expected, this is such a common pattern (the usage of `match...with`) that we have a shortcut for it:

In [29]:
let concat ?(sep="") x y =
  x ^ sep ^ y

val concat : ?sep:string -> string -> string -> string = <fun>
